In [18]:
!pip install datasets
!pip install -U bitsandbytes

In [2]:
import argparse
import json
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path
import matplotlib.pyplot as plt
import os
import numpy as np
import datasets
import torch
import transformers
from datasets import load_dataset
from datasets import load_from_disk
from huggingface_hub import Repository
import json
import math
import random
import logging
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, get_scheduler
from accelerate import Accelerator
from transformers.utils import get_full_repo_name
from transformers.utils.versions import require_version
import accelerate

from accelerate import Accelerator, DistributedType
from accelerate.logging import get_logger
# from accelerate.utils import DummyOptim, DummyScheduler, set_seed
import torch
from torch.optim import Optimizer
from transformers import get_scheduler

class DummyOptimizer(Optimizer):
    def __init__(self, params):
        super().__init__(params, {})

    def step(self, closure=None):
        pass

    def zero_grad(self):
        pass
logger = get_logger(__name__)
require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")
from transformers import AutoConfig, MODEL_MAPPING  # Import MODEL_MAPPING
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)
from accelerate import Accelerator

# Enable mixed precision with fp16
accelerator = Accelerator()
from transformers import SchedulerType  # Import SchedulerType from transformers
import pandas as pd
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
import pandas as pd

In [16]:
device = "cuda"

In [17]:
model = AutoModelForCausalLM.from_pretrained("qu-bit/SuperLLM", load_in_4bit=True, device_map="auto")
# Load dataset from CSV
df = pd.read_csv('/content/harrypottercsv.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [18]:
tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")
# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [19]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator


# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)
# Use DataLoader to create the dataloader and specify the batch size here

In [20]:
import argparse
from transformers import get_scheduler
from torch.optim import AdamW

# Define a dummy scheduler in case DeepSpeed manages the scheduler
class DummyScheduler:
    def __init__(self, optimizer, total_num_steps, warmup_num_steps):
        self.optimizer = optimizer
        self.total_num_steps = total_num_steps
        self.warmup_num_steps = warmup_num_steps

    def step(self):
        pass  # No-op for DummyScheduler

# Argument parser setup
parser = argparse.ArgumentParser()

parser.add_argument(
    "--lr_scheduler_type",
    type=str,
    default="linear",
    help="The scheduler type to use.",
    choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"],
)

parser.add_argument(
    "--num_warmup_steps",
    type=int,
    default=0,
    help="Number of steps for the warmup phase in the learning rate scheduler."
)

parser.add_argument(
    "--max_train_steps",
    type=int,
    default=1000,
    help="Total number of training steps."
)

args = parser.parse_args([])

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the learning rate scheduler based on whether DeepSpeed is used or not
if (
    accelerator.state.deepspeed_plugin is None
    or "scheduler" not in accelerator.state.deepspeed_plugin.deepspeed_config
):
    # If DeepSpeed is not used or it doesn't manage the scheduler, use the defined scheduler
    lr_scheduler = get_scheduler(
        name=args.lr_scheduler_type,
        optimizer=optimizer,
        num_warmup_steps=args.num_warmup_steps,
        num_training_steps=args.max_train_steps,
    )
else:
    # If DeepSpeed manages the scheduler, use the DummyScheduler
    lr_scheduler = DummyScheduler(
        optimizer, total_num_steps=args.max_train_steps, warmup_num_steps=args.num_warmup_steps
    )

# Training loop
for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed
Step 5 completed
Step 6 completed
Step 7 completed
Step 8 completed
Step 9 completed
Step 10 completed
Step 11 completed
Step 12 completed
Step 13 completed
Step 14 completed
Step 15 completed
Step 16 completed
Step 17 completed
Step 18 completed
Step 19 completed
Step 20 completed
Step 21 completed
Step 22 completed
Step 23 completed
Step 24 completed
Step 25 completed
Step 26 completed
Step 27 completed
Step 28 completed
Step 29 completed
Step 30 completed
Step 31 completed
Step 32 completed
Step 33 completed
Step 34 completed
Step 35 completed
Step 36 completed
Step 37 completed
Step 38 completed
Step 39 completed
Step 40 completed
Step 41 completed
Step 42 completed
Step 43 completed
Step 44 completed
Step 45 completed
Step 46 completed
Step 47 completed
Step 48 completed
Step 49 completed
Step 50 completed
Step 51 completed
Step 52 completed
Step 53 completed
Step 54 completed
Step 55 completed
St

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
df = pd.read_csv('/content/percyjackson.csv')

In [25]:
tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [26]:
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

In [27]:
for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed
Step 5 completed
Step 6 completed
Step 7 completed
Step 8 completed
Step 9 completed
Step 10 completed
Step 11 completed
Step 12 completed
Step 13 completed
Step 14 completed
Step 15 completed
Step 16 completed
Step 17 completed
Step 18 completed
Step 19 completed
Step 20 completed
Step 21 completed
Step 22 completed
Step 23 completed
Step 24 completed
Step 25 completed
Step 26 completed
Step 27 completed
Step 28 completed
Step 29 completed
Step 30 completed
Step 31 completed
Step 32 completed
Step 33 completed
Step 34 completed
Step 35 completed
Step 36 completed
Step 37 completed
Step 38 completed
Step 39 completed
Step 40 completed
Step 41 completed
Step 42 completed
Step 43 completed
Step 44 completed
Step 45 completed
Step 46 completed
Step 47 completed
Step 48 completed
Step 49 completed
Step 50 completed
Step 51 completed
Step 52 completed
Step 53 completed
Step 54 completed
Step 55 completed
St

In [28]:
df = pd.read_csv('/content/taylor swift.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed


In [29]:
df = pd.read_csv('/content/anakin.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed


In [31]:
df = pd.read_csv('/content/boris_johnson.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed


In [32]:
df = pd.read_csv('/content/charlie_chaplin.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed


In [33]:
df = pd.read_csv('/content/indira .csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed
Step 5 completed


In [34]:
df = pd.read_csv('/content/dicaprio final.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed


In [35]:
df = pd.read_csv('/content/sania_nehwal.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [36]:
df = pd.read_csv('/content/rhianna.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed


In [37]:
df = pd.read_csv('/content/rk narayan.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [38]:
df = pd.read_csv('/content/rahul gandhi.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [39]:
df = pd.read_csv('/content/serena_williams.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed


In [40]:
df = pd.read_csv('/content/simon.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed


In [41]:
df = pd.read_csv('/content/shreya ghoshal.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed


In [42]:
df = pd.read_csv('/content/putin.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed
Step 5 completed
Step 6 completed


In [43]:
df = pd.read_csv('/content/narendra modi.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed


In [44]:
df = pd.read_csv('/content/donaldTrump.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed
Step 5 completed


In [45]:
df = pd.read_csv('/content/lewis hamilton final.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed
Step 3 completed
Step 4 completed


In [46]:
df = pd.read_csv('/content/mamata.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [47]:
df = pd.read_csv('/content/deepika.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [48]:
df = pd.read_csv('/content/aishwaryaRai.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed


In [49]:
df = pd.read_csv('/content/jude.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed


In [50]:
df = pd.read_csv('/content/kohli.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [51]:
df = pd.read_csv('/content/marryKOMm.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Step 0 completed


In [52]:
df = pd.read_csv('/content/neeraj.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [53]:
df = pd.read_csv('/content/sudha.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Step 0 completed


In [54]:
df = pd.read_csv('/content/satya_nadella.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [55]:
df = pd.read_csv('/content/rani.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed
Step 2 completed


In [57]:
df = pd.read_csv('/content/kalpna.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [59]:
df = pd.read_csv('/content/marrie curie.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [60]:
df = pd.read_csv('/content/sourav ghoshal.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Step 0 completed


In [61]:
df = pd.read_csv('/content/yusuf.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Step 0 completed


In [63]:
df = pd.read_csv('/content/mukesh ambani.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [64]:
df = pd.read_csv('/content/james.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Step 0 completed
Step 1 completed


In [65]:
df = pd.read_csv('/content/ruskin.csv')

tokenizer = AutoTokenizer.from_pretrained("qu-bit/SuperLLM")

# Tokenize data
def preprocess_data(examples):
    # Join the list of tokens into a single string for each example
    examples['tokens'] = [' '.join(tokens) for tokens in examples['tokens']]
    return tokenizer(
        examples['tokens'],
        padding="max_length",
        truncation=True,
        max_length=512,  # Set the maximum length here
        return_tensors="pt"
    )

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]

from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from accelerate import Accelerator

# ... your existing code ...

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=default_data_collator)

for step, batch in enumerate(train_dataloader):
    with accelerator.accumulate(model):
        # Move batch data to the device
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Use accelerator.device

        outputs = model(**batch)
        # Extract the loss value from the outputs and calculate the mean
        loss = outputs['logits'].mean()  # Adjust this line based on your model's output structure
        accelerator.backward(loss)

        # Step optimizer and scheduler
        optimizer.step()
        lr_scheduler.step()  # Update the learning rate
        optimizer.zero_grad()

    print(f"Step {step} completed")

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Step 0 completed


In [68]:
# model.generation_config.do_sample = True
model.push_to_hub("AryanSuperLLM")
tokenizer.push_to_hub("AryanSuperLLM")

model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ProElectro07/AryanSuperLLM/commit/6bb9d13d3dcf41a1f7cf99ff0a14831ce00c3866', commit_message='Upload tokenizer', commit_description='', oid='6bb9d13d3dcf41a1f7cf99ff0a14831ce00c3866', pr_url=None, pr_revision=None, pr_num=None)